<a href="https://colab.research.google.com/github/lida-3/Exploratory-data-analysis-with-natural-language-text-processing/blob/main/done_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import numpy as np
import pandas as pd
import requests
import bs4
import lxml.etree as xml
import pandas as pd
import matplotlib

from google.colab import files

In [ ]:
pp_links=[]
for i in range(2022,2021,-1):
  pp_links.append('https://www.morh.hr/kategorija/protupozarne-aktivnosti-'+str(i)+'/page/')
data_all=[]
base='https://www.morh.hr'
for url_base in pp_links:

  for page in range(13):
    WP=url_base+str(page)+'/'
    web_page = bs4.BeautifulSoup(requests.get(WP, headers={   "UserAgent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.183 Safari/537.36"}).text, "lxml")
    primary=web_page.find(id='primary')
    #fd-c-media entry-preview post-84212 post type-post status-publish format-standard has-post-thumbnail hentry category-priopcenja category-protupozarne-aktivnosti-2018
    #fd-c-media entry-preview post-202792 post type-post status-publish format-standard has-post-thumbnail hentry category-priopcenja category-protupozarne-aktivnosti-2022
    for newsi in primary.find_all('li'):
      try:
        meta1=newsi.select('.entry-preview__meta')
        date=newsi.select('.date')[0].text.strip()
        tt=newsi.select('.entry-preview__summary')[0].text.strip()
        title=newsi.select('.entry-preview__title')[0]
        link=title.find('a')
        post1={'datestr':date, 'title':link.text.strip(),'href':link['href'],'summary':tt }
        data_all.append(post1)
      except Exception as e:
        print(e)


In [ ]:
df=pd.DataFrame(data=data_all)

bodies=[]
for ull_a in df['href']:

  txt=""
  article_page = bs4.BeautifulSoup(requests.get(ull_a, headers={"UserAgent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.183 Safari/537.36"}).text, "lxml")
  #for b in article_page.select('.entry__body'):
  for b in article_page.select('.entry__content'):
    txt=txt + b.text.strip()
  bodies.append(txt)
df['bodies']=bodies

In [ ]:
templates =  {"total_requests": ".*Ukupno je u 2022. zaprimljeno (?P<total_requests>[0-9]+) zahtjev.* .*{1}",
              "total_number_flights": "{2} (?P<total_number_flights>(([0-9]+\.[0-9]+)|[0-9]+)) let.* {3}",
              "total_hours_flight": "{3} (?P<total_hours_flight>(([0-9]+)|([0-9]+\.[0-9]+)|[0-9]+:[0-9]+:[0-9]+)|([0-9]+:[0-9]+)) sat.* nalet.* .* {4}",
              "total_tons_water": "{4} (?P<total_tons_water>(([0-9]+\.[0-9]+)|[0-9]+)) ton. vod.*(\.*|,* .*){5}"
              }


In [ ]:
data_dict =  {
             "total_requests": [],
             "total_number_flights": [],
             "total_hours_flight": [],
             "total_tons_water": []
}

In [ ]:
# pronalazi informacije o trenutnom požaru
def find_template(input_text):

    for template_key, template in templates.items():
      regex_temp = re.sub(r'\{\s*\d+\s*\}','.+',template) + '$'
      z = re.match(regex_temp, input_text, flags=re.I | re.MULTILINE | re.DOTALL)

      if z:
      # print(template_key)
        #print("groupdict",z.groupdict())
        if template_key == "total_requests":
          data_dict["total_requests"].append(z.groupdict()['total_requests'])
        elif template_key == "total_number_flights":
          data_dict["total_number_flights"].append(z.groupdict()['total_number_flights'])
        elif template_key == "total_hours_flight":
          data_dict["total_hours_flight"].append(z.groupdict()['total_hours_flight'])
        elif template_key == "total_tons_water":
          data_dict["total_tons_water"].append(z.groupdict()['total_tons_water'])

      else:
        if template_key == "total_requests":
          data_dict["total_requests"].append("")
        elif template_key == "total_number_flights":
          data_dict["total_number_flights"].append("")
        elif template_key == "total_hours_flight":
          data_dict["total_hours_flight"].append("")
        elif template_key == "total_tons_water":
          data_dict["total_tons_water"].append("")


In [ ]:
for row in df['bodies']:
  find_template(row)

print(data_dict)

{'total_requests': ['', '192', '', '', '', '', '', '', '', '', '', '192', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '55', '49', '', '', '', ''], 'total_number_flights': ['6.600', '4303', '3664', '', '3022', '', '72', '72', '', '', '6.600', '4303', '3664', '', '3022', '', '72', '72', '', '', '', '', '2654', '', '', '1212', '214', '', '971', '907', '', '', '', ''], 'total_hours_flight': ['996', '643.36', '557', '', '446', '', '', '', '', '', '996', '643.36', '557', '', '446', '', '', '', '', '', '', '', '372', '', '', '169', '44:25', '154', '159:30', '136:05', '', '', '', ''], 'total_tons_water': ['32.714', '18.699', '18.303', '', '15.175', '', '222', '222', '', '48', '32.714', '18.699', '18.303', '', '15.175', '', '222', '222', '', '48', '', '', '', '', '', '374', '1045', '4073', '', '4.231', '', '', '', '']}


In [ ]:
data_dict2 = {
    "date": [],
    "text": []
}

In [ ]:
for row in df['bodies']:
  data_dict2["date"] = df['datestr']
  data_dict2["text"] = df['bodies']


In [ ]:
df_data = pd.DataFrame(data_dict)
df_data


,total_requests,total_number_flights,total_hours_flight,total_tons_water
0,,6.600,996,32.714
1,192,4303,643.36,18.699
2,,3664,557,18.303
3,,,,
4,,3022,446,15.175
5,,,,
6,,72,,222
7,,72,,222
8,,,,
9,,,,48


In [ ]:
df_data2 = pd.DataFrame(data_dict2)
df_data2

,date,text
0,19.09.2022.,Glavni vatrogasni zapovjednik i ministar obran...
1,08.08.2022.,Od početka II. faze protupožarne sezone pa do ...
2,01.08.2022.,Zračne snage Protupožarnih namjenski organizir...
3,26.07.2022.,Jedan helikopter Mi-8 MTV Hrvatskog ratnog zra...
4,25.07.2022.,"Od početka II. faze protupožarne sezone, od 13..."
5,23.07.2022.,"Požar u Sloveniji i dalje je aktivan, jutros, ..."
6,22.07.2022.,Helikopter Mi-8 MTV s opremom za gašenje požar...
7,22.07.2022.,Helikopter Mi-8 MTV s opremom za gašenje požar...
8,22.07.2022.,Canadair CL-415 iz sastava Protupožarnih Namje...
9,21.07.2022.,Helikopter Mi-8 MTV s opremom za gašenje požar...


In [ ]:
complete = pd.concat([df_data2, df_data], axis="columns")
complete


,date,text,total_requests,total_number_flights,total_hours_flight,total_tons_water
0,19.09.2022.,Glavni vatrogasni zapovjednik i ministar obran...,,6.600,996,32.714
1,08.08.2022.,Od početka II. faze protupožarne sezone pa do ...,192,4303,643.36,18.699
2,01.08.2022.,Zračne snage Protupožarnih namjenski organizir...,,3664,557,18.303
3,26.07.2022.,Jedan helikopter Mi-8 MTV Hrvatskog ratnog zra...,,,,
4,25.07.2022.,"Od početka II. faze protupožarne sezone, od 13...",,3022,446,15.175
5,23.07.2022.,"Požar u Sloveniji i dalje je aktivan, jutros, ...",,,,
6,22.07.2022.,Helikopter Mi-8 MTV s opremom za gašenje požar...,,72,,222
7,22.07.2022.,Helikopter Mi-8 MTV s opremom za gašenje požar...,,72,,222
8,22.07.2022.,Canadair CL-415 iz sastava Protupožarnih Namje...,,,,
9,21.07.2022.,Helikopter Mi-8 MTV s opremom za gašenje požar...,,,,48


In [ ]:
complete.to_excel("2022_data.xlsx")
files.download('2022_data.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>